In [1]:
import math
import time
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def load_data(path):
    df = pd.read_csv(path)
    data = df.drop(columns=['Unnamed: 0'])
    
    return data

In [3]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    return trainX, trainY, testX, testY

In [4]:
def getTestY(data):
    tmp_scaler = StandardScaler()
    tmp_data = tmp_scaler.fit_transform(data)
    tmp_data = tmp_scaler.inverse_transform(tmp_data)
    _, _, _, testY = preprocess_data(tmp_data, data.shape[0], train_rate, seq_len, pre_len)
    testY = np.array(testY)
    testY = np.reshape(testY, [-1, pre_len])
    
    return testY

In [5]:
 def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    mape = mean_absolute_percentage_error(a, b)
    F_norm = la.norm(a-b)/la.norm(a)
    
    return rmse, mae, mape, 1-F_norm

In [6]:
def predictSVR(data, testY):
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    
    rmses, maes, mapes, accs = [], [], [], []
    for i in range(data.shape[1]):
        print('Node', i)
        start = time.time()
        a = data[:, i]
        aX, aY, tX, tY = preprocess_data(a, data.shape[0], train_rate, seq_len, pre_len)
        
        aX = np.array(aX)
        aX = np.reshape(aX, [-1, seq_len])
        aY = np.array(aY)
        aY = np.reshape(aY, [-1, pre_len])

        tX = np.array(tX)
        tX = np.reshape(tX, [-1, seq_len])
        tY = np.array(tY)
        tY = np.reshape(tY, [-1, pre_len])
        
        print('(', aX.shape, aY.shape, tX.shape, tY.shape, round(aX.mean(),3), ')')
        reg = MultiOutputRegressor(SVR(kernel='linear'))
        reg.fit(aX, aY)
        pred = reg.predict(tX)
        
        mean = scaler.mean_[i]
        std = np.sqrt(scaler.var_[i])
        pred = pred*std + mean
        tY = tY*std + mean
        if i==data.shape[1]-1: tY = testY[tY.shape[0]*i:]
        else: tY = testY[tY.shape[0] * i:tY.shape[0] * (i+1)]
        
        rmse, mae, mape, acc = evaluation(tY, pred)
        rmses.append(rmse)
        maes.append(mae)
        mapes.append(mape)
        accs.append(acc)
        
        print('(', rmse, mae, mape, acc, time.time() - start)
    
    print('RMSE: ' + str(np.mean(rmses)) + ', MAE: ' + str(np.mean(maes)) + ', MAPE: ' + str(np.mean(mapes)) + ', ACC: ' + str(np.mean(accs)))

In [7]:
train_rate = 0.8
seq_len = 12
pre_len = 9

In [8]:
path_0 = '../Data/Gangnam/speed_gangnam_0.csv'
path_5 = '../Data/Gangnam/speed_gangnam_5.csv'
path_10 = '../Data/Gangnam/speed_gangnam_10.csv'
path_20 = '../Data/Gangnam/speed_gangnam_20.csv'

In [9]:
masterData = load_data(path_0)
print(masterData.shape, (masterData == 0).sum().sum())
testY = getTestY(masterData)
testY.shape, (testY == 0).sum().sum()

(2880, 506) 0


((280830, 9), 0)

### Unobserved Nodes = 0%

In [10]:
data = load_data(path_0)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 0
Node 0
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.004 )
( 11.00879504635314 7.772793263667448 0.25142900372765753 0.659033748534849 3.817795753479004
Node 1
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.032 )
( 13.44887816097216 9.978754071302863 0.3205404511624257 0.583641153296564 3.434849739074707
Node 2
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.096 )
( 12.112873572878746 9.403821001567607 0.3383667926090826 0.6272416106475702 3.7329936027526855
Node 3
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.001 )
( 14.765205598834502 12.587014549349643 0.5003209795344729 0.5486158415553315 4.028234004974365
Node 4
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.005 )
( 16.602768363071924 12.93213586496207 0.38286114961964934 0.4933663565319828 4.0850818157196045
Node 5
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.019 )
( 13.593867096103622 9.84779152496666 0.2833736338001243 0.5858874527416509 5.001633882522583
Node 6
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.013 )
( 11.926909577

( 19.1658569361172 16.114094261344988 0.46669870520596457 0.4220089265752248 4.424173593521118
Node 54
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.009 )
( 17.440205106290808 14.15330784735362 0.4047056125052084 0.47280439688388687 4.702435255050659
Node 55
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.008 )
( 10.74000064393313 7.724924876898261 0.2608809003178196 0.6758491491887426 5.914191246032715
Node 56
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.003 )
( 13.248889678160463 10.66300705310276 0.42002222822759566 0.599071456510937 4.759280443191528
Node 57
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.007 )
( 11.687795472184094 8.584593299251534 0.29495755571830135 0.646910905160211 4.689467191696167
Node 58
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.037 )
( 15.745457257126677 13.313479837301816 0.5367957535186636 0.5242356129833974 4.2785656452178955
Node 59
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 11.926683316442777 9.231334640932898 0.3540439782014514 0.6389747806609654 4.

( 13.390315083867279 11.773155535767351 0.6001696418002818 0.5048135866394478 4.731354236602783
Node 108
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.018 )
( 9.891327932096639 7.073211911563055 0.29089031216663214 0.6340387480559382 5.53624701499939
Node 109
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.011 )
( 10.541319101430416 7.9055531492616495 0.34230302704376286 0.6082356796110826 4.717347621917725
Node 110
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.033 )
( 10.218981076468387 7.891528374731436 0.3892594461046709 0.6183164659103397 5.223039388656616
Node 111
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.017 )
( 9.607837110677309 6.963185813864321 0.3088826332542267 0.6398419623596217 4.415243148803711
Node 112
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.026 )
( 9.920102338883943 7.485684067717248 0.36001547341575 0.628905621301862 4.47898530960083
Node 113
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.003 )
( 11.008931760657797 8.332846177960342 0.3652965591688283 0.5874331015166379 4

( 10.108639755122706 7.451951490358222 0.37528365140421843 0.6154608222415126 4.758281946182251
Node 162
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.004 )
( 12.221225643035663 9.050287570112117 0.3985389414348086 0.5321895169848356 4.049176454544067
Node 163
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.027 )
( 10.350521848964265 7.660120516310063 0.3902656247370104 0.6049397282781788 4.123978614807129
Node 164
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.003 )
( 12.478375822359618 9.727614655045352 0.5184510310556902 0.5242533082104666 4.934812784194946
Node 165
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.006 )
( 11.816005560877825 9.379821021312987 0.49013049159803035 0.550523854380967 4.353365182876587
Node 166
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.063 )
( 17.63074232781005 14.60960164871344 0.7647583988459928 0.3285763712380878 3.2692620754241943
Node 167
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.059 )
( 17.79038076113053 14.485233209051469 0.8054546975875366 0.3207434697658

( 12.263581548625613 8.810275335388743 0.29224022673692884 0.5778634943280164 4.0641701221466064
Node 216
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.066 )
( 10.19796039866143 7.2528062276099865 0.2849990912464422 0.6485726956851527 4.577732086181641
Node 217
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.019 )
( 18.00771406510125 16.28087125098718 0.7606820733052845 0.3815294615890431 4.653564691543579
Node 218
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.055 )
( 11.848354539233984 8.565890809470245 0.31168658348479195 0.5949161804427817 4.0532097816467285
Node 219
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.027 )
( 12.267100833268687 9.916385620350512 0.44893303450362065 0.5778709676000227 4.440088987350464
Node 220
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.006 )
( 12.812599382177378 10.242096128245702 0.456613689163892 0.560615160781428 4.679494857788086
Node 221
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.016 )
( 10.478263659839774 8.079355824083338 0.3569211805697576 0.641971723413

( 13.39103147061515 9.685920749899825 0.28007182765520994 0.602295888069593 4.855024814605713
Node 269
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.006 )
( 12.700543010497555 9.27388019562424 0.29256141238741995 0.6229542442577335 6.410700798034668
Node 270
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.035 )
( 20.653259142264787 17.213975087859872 0.49618903331367814 0.3884880239205448 4.8898844718933105
Node 271
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.002 )
( 13.270977918970116 9.755029893883028 0.31289452891476704 0.6080082491549201 5.280919075012207
Node 272
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.005 )
( 11.803378664677174 8.517581641423767 0.2761140680012712 0.6520529861214923 5.703917741775513
Node 273
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.005 )
( 13.71078343832615 10.168852533621028 0.3022662636834981 0.5966618551139702 5.12932825088501
Node 274
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.001 )
( 16.502817296703565 12.821777655777785 0.36312671202740876 0.514073213

( 12.473797445901582 9.19455863146002 0.2969058664642214 0.6238270676602482 4.676531553268433
Node 322
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.007 )
( 13.417708071049306 10.154979775784291 0.33915313445589557 0.5957432989969604 3.5395424365997314
Node 323
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.009 )
( 15.204174220182336 11.599996167234536 0.3506111365204832 0.5419529490612044 3.791834592819214
Node 324
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.021 )
( 12.5483017583832 9.65893377561396 0.354704781670801 0.620886405857896 4.254469156265259
Node 325
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.033 )
( 12.007950398606468 9.457219025241528 0.3792837322750526 0.6361802714935496 4.951794862747192
Node 326
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.027 )
( 11.201452025637689 7.990635444673164 0.2812362012623776 0.6612422666805775 4.228502511978149
Node 327
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.07 )
( 16.772639965197595 14.07559201100798 0.5841124556579632 0.49244941920036933 5

( 15.377362816364263 13.285172225456673 0.6921840095840017 0.42763821926995693 4.885042428970337
Node 375
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.013 )
( 38.384763506179006 36.353057969137446 1.8012288047377234 -0.43211930524586983 3.549542188644409
Node 376
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 58.35817679400372 57.509533490153395 2.779741009282079 -1.1767969744186129 3.6702096462249756
Node 377
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.034 )
( 42.34593000047443 38.796441348172976 1.9442190942023931 -0.5843811798777385 2.563100576400757
Node 378
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.001 )
( 33.735091667436535 30.531947087463802 1.56200172910767 -0.26672620460601304 2.9351558685302734
Node 379
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.004 )
( 11.031265227241473 8.387005139592857 0.41131661919137597 0.5845556565626895 4.370320081710815
Node 380
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.05 )
( 12.436329552916382 10.216503543574456 0.5468272041603255 0.52998

( 28.000233287940997 25.389072254456874 1.4314158522776903 -0.08833242341877034 3.1116864681243896
Node 428
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.015 )
( 32.728813132862165 30.533967934466514 1.707296698295106 -0.26846036455000055 3.699113607406616
Node 429
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.012 )
( 34.87894288083938 32.0867851658784 1.8130467212511003 -0.3606602707345854 2.8520286083221436
Node 430
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.028 )
( 38.19826087323378 36.78762511560589 2.1048113725285607 -0.49744562665272873 4.442665338516235
Node 431
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.004 )
( 27.901759156151005 26.155544303716 1.5274711496708349 -0.09083195052674542 5.4331278800964355
Node 432
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.026 )
( 12.04265846224838 8.653151366246187 0.35149254583951617 0.5272202718441652 4.171835422515869
Node 433
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.01 )
( 14.691700177118937 11.399216431401134 0.43883366098084925 0.42171

( 10.068771703667997 6.965346700049616 0.2493469478944735 0.6591087940187021 4.7991626262664795
Node 482
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.084 )
( 16.25596043452029 13.110617729489173 0.4276388634560048 0.4505400110745901 4.441098690032959
Node 483
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.049 )
( 10.827411399041473 7.709489228758078 0.2712515293860586 0.6334153732831245 4.0491783618927
Node 484
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.011 )
( 10.783464366823608 7.906142950084424 0.2917934499025334 0.6350328779490876 4.40223503112793
Node 485
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.014 )
( 9.82357021596864 7.073821506948174 0.27462019837627627 0.6675416004644458 5.291904449462891
Node 486
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.027 )
( 10.951886035841271 7.811622829161915 0.2854295584253249 0.6301717787774342 4.225691318511963
Node 487
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.054 )
( 14.405234528675956 12.690263955564236 0.5776764004523434 0.511319037416375 

### Unobserved Nodes = 5%

In [11]:
data = load_data(path_5)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 72000
Node 0
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.004 )
( 11.00879504635314 7.772793263667448 0.25142900372765753 0.659033748534849 4.4122395515441895
Node 1
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.032 )
( 13.44887816097216 9.978754071302863 0.3205404511624257 0.583641153296564 3.6841559410095215
Node 2
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.096 )
( 12.112873572878746 9.403821001567607 0.3383667926090826 0.6272416106475702 3.9284675121307373
Node 3
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.001 )
( 14.765205598834502 12.587014549349643 0.5003209795344729 0.5486158415553315 4.109018087387085
Node 4
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.005 )
( 16.602768363071924 12.93213586496207 0.38286114961964934 0.4933663565319828 4.140965938568115
Node 5
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.019 )
( 13.593867096103622 9.84779152496666 0.2833736338001243 0.5858874527416509 4.705392837524414
Node 6
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.013 )
( 11.9269

( 17.440205106290808 14.15330784735362 0.4047056125052084 0.47280439688388687 4.2007737159729
Node 55
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.008 )
( 10.74000064393313 7.724924876898261 0.2608809003178196 0.6758491491887426 5.2001330852508545
Node 56
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.003 )
( 13.248889678160463 10.66300705310276 0.42002222822759566 0.599071456510937 4.0870466232299805
Node 57
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.007 )
( 11.687795472184094 8.584593299251534 0.29495755571830135 0.646910905160211 4.211792230606079
Node 58
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.037 )
( 15.745457257126677 13.313479837301816 0.5367957535186636 0.5242356129833974 3.77586030960083
Node 59
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 11.926683316442777 9.231334640932898 0.3540439782014514 0.6389747806609654 4.3005359172821045
Node 60
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.052 )
( 11.436450741607654 8.825940455518738 0.34243602479325563 0.6534210566854605 3

( 10.541319101430416 7.9055531492616495 0.34230302704376286 0.6082356796110826 4.160873889923096
Node 110
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.033 )
( 10.218981076468387 7.891528374731436 0.3892594461046709 0.6183164659103397 4.558810234069824
Node 111
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.017 )
( 9.607837110677309 6.963185813864321 0.3088826332542267 0.6398419623596217 3.8736441135406494
Node 112
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.026 )
( 9.920102338883943 7.485684067717248 0.36001547341575 0.628905621301862 4.059155464172363
Node 113
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.003 )
( 11.008931760657797 8.332846177960342 0.3652965591688283 0.5874331015166379 3.588378667831421
Node 114
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.04 )
( 9.207109602295144 6.7560164438374875 0.3344353805041866 0.6548557526277855 4.900933742523193
Node 115
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.017 )
( 10.340618049086881 7.769600237171435 0.3780780146802873 0.6098679850739375 4

( 10.350521848964265 7.660120516310063 0.3902656247370104 0.6049397282781788 3.797842025756836
Node 164
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.003 )
( 12.478375822359618 9.727614655045352 0.5184510310556902 0.5242533082104666 4.457089424133301
Node 165
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.006 )
( 11.816005560877825 9.379821021312987 0.49013049159803035 0.550523854380967 3.6402676105499268
Node 166
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.063 )
( 17.63074232781005 14.60960164871344 0.7647583988459928 0.3285763712380878 2.9282066822052
Node 167
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.059 )
( 17.79038076113053 14.485233209051469 0.8054546975875366 0.32074346976587087 3.24534273147583
Node 168
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.004 )
( 29.031761667525963 27.349629860818514 1.4609562941342231 -0.10337818475828664 4.246631622314453
Node 169
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.008 )
( 16.014866917981127 13.163457289147814 0.6470670961388605 0.391204662681

( 18.00771406510125 16.28087125098718 0.7606820733052845 0.3815294615890431 4.200743198394775
Node 218
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.055 )
( 11.848354539233984 8.565890809470245 0.31168658348479195 0.5949161804427817 3.6452574729919434
Node 219
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.027 )
( 12.267100833268687 9.916385620350512 0.44893303450362065 0.5778709676000227 3.7808942794799805
Node 220
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.006 )
( 12.812599382177378 10.242096128245702 0.456613689163892 0.560615160781428 3.674180030822754
Node 221
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.016 )
( 10.478263659839774 8.079355824083338 0.3569211805697576 0.6419717234133249 4.097051620483398
Node 222
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.016 )
( 10.211636829291757 7.226835439974997 0.27512683350456757 0.6507602048100007 4.047184228897095
Node 223
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.024 )
( 10.662860809467945 7.833127059490489 0.31041074604186814 0.6349809072

( 13.270977918970116 9.755029893883028 0.31289452891476704 0.6080082491549201 4.202736854553223
Node 272
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.005 )
( 11.803378664677174 8.517581641423767 0.2761140680012712 0.6520529861214923 4.601701021194458
Node 273
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 33.99327242455675 32.14050050050051 1.0 0.0 0.009972810745239258
Node 274
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.001 )
( 16.502817296703565 12.821777655777785 0.36312671202740876 0.5140732130848122 3.700143337249756
Node 275
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.024 )
( 15.982534691977905 12.265542055940434 0.3484215016519858 0.5309986692470254 3.7170655727386475
Node 276
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.032 )
( 17.11298580041782 13.443089488856852 0.38561087815386413 0.4980147606953824 3.5395429134368896
Node 277
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.131 )
( 18.15327471405322 14.587166430122485 0.40851420396616267 0.4686354786943805 3.978362798690796
N

( 12.007950398606468 9.457219025241528 0.3792837322750526 0.6361802714935496 4.143924951553345
Node 326
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.027 )
( 11.201452025637689 7.990635444673164 0.2812362012623776 0.6612422666805775 3.6861183643341064
Node 327
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.07 )
( 16.772639965197595 14.07559201100798 0.5841124556579632 0.49244941920036933 4.778228759765625
Node 328
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 32.861592542425946 31.043371371371368 1.0 0.0 0.012000322341918945
Node 329
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.046 )
( 12.524689307774034 9.537480206303387 0.34906200906060914 0.6186317969131427 4.441099643707275
Node 330
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.044 )
( 11.62366277729959 8.724301199141989 0.3299300201308191 0.6466265162323714 4.402233839035034
Node 331
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.008 )
( 12.361441567088521 9.923929994056989 0.4071731640015793 0.6220221313660099 3.5943944454193115
Node 33

( 11.031265227241473 8.387005139592857 0.41131661919137597 0.5845556565626895 3.7489829063415527
Node 380
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.05 )
( 12.436329552916382 10.216503543574456 0.5468272041603255 0.529988021366572 3.6711597442626953
Node 381
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.034 )
( 11.19509105379187 8.461111373773061 0.40532657135055855 0.5758025057896259 3.7230570316314697
Node 382
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 26.292278436555957 24.430338338338338 1.0 0.0 0.008971929550170898
Node 383
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.032 )
( 10.231899202225678 7.66304486375377 0.3985926592117603 0.6106671850814569 3.8257787227630615
Node 384
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.055 )
( 10.421060617894724 7.676714561282817 0.37081223185483236 0.6035983635384996 3.628298759460449
Node 385
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.041 )
( 10.09230896881532 7.460740894098279 0.38028486196825745 0.6153556575634073 4.75426459312439
Node 

( 14.691700177118937 11.399216431401134 0.43883366098084925 0.42171432178903856 3.428802967071533
Node 434
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.0 )
( 25.259008007947294 23.3325985985986 1.0 0.0 0.008975505828857422
Node 435
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.07 )
( 10.608732964175173 7.653503810025703 0.40093101123645236 0.5785759873242251 3.465841293334961
Node 436
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.012 )
( 11.367165281818206 8.132502017430454 0.3373563239505297 0.5518926622311293 3.282228946685791
Node 437
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.018 )
( 10.787851465348126 8.113071788266819 0.4250033607299399 0.5724906701223402 2.816472291946411
Node 438
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.019 )
( 10.171861626421089 7.843729479510998 0.4464681854676756 0.5978353280046209 4.082129001617432
Node 439
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.023 )
( 11.421028375919404 9.361424752269322 0.561427785291601 0.5481412560556506 4.043185472488403
Node 4

( 10.579793912038303 8.280569510576957 0.361125204667164 0.6411848210682041 4.067128896713257
Node 489
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.028 )
( 17.41677560935944 15.73430083354041 0.7191403771835335 0.4076493703357854 3.4178645610809326
Node 490
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.027 )
( 14.943627259463483 13.149629832392321 0.6108727082501817 0.49153155627790224 4.542860269546509
Node 491
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.025 )
( 10.446463100416048 7.511224514813544 0.2869652596759124 0.6435876385069619 3.414872169494629
Node 492
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.012 )
( 12.3826370077743 9.067465394156288 0.31304871992474614 0.5774866401093887 3.8068268299102783
Node 493
( (2283, 12) (2283, 9) (555, 12) (555, 9) -0.017 )
( 10.437327037487506 7.997503751386688 0.34258606769031413 0.6456430316978146 4.559812307357788
Node 494
( (2283, 12) (2283, 9) (555, 12) (555, 9) 0.009 )
( 11.63736562971134 8.520503164715684 0.29967826643686846 0.6074018010